<a href="https://colab.research.google.com/github/AlexAlexeyuk/Alexander.Alexeyuk/blob/master/stat_ml_begining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import pandas as pd
import numpy as np
from scipy import stats
from scipy.stats import mannwhitneyu,ttest_ind, kruskal,chisquare, pearsonr, spearmanr

In [2]:
df = pd.read_csv('/content/covid_data_6_mistakes (1).csv')

In [3]:
columns = df.columns

In [9]:
def Basic_statistics(lst_1, lst_2):

    res_tt = ttest_ind(lst_1, lst_2) # т-тест стьюдента
    res_mw = mannwhitneyu(lst_1, lst_2, 
                       use_continuity=True, 
                       alternative=None) # u -тест манна-уитни
    res_kw = kruskal(lst_1, lst_2)## краскела уолиса
    res_chi = chisquare(lst_1, lst_2) ## хи-квадрат
    res_pearson = pearsonr(lst_1, lst_2)# корреляция пирсона
    res_spear = spearmanr(lst_1, lst_2)# корреляция спирмана
    res1_ks = stats.kstest(lst_1,'norm',alternative='greater')#тест колмагорова-смирнова
    res2_ks = stats.kstest(lst_2,'norm',alternative='greater')#тест колмагорова-смирнова
    shapiro_test = stats.shapiro(lst_2)#тест шапиро-уилка
    anova = stats.f_oneway(lst_1, lst_2)
    oddsratio, pvalue = stats.fisher_exact([[8, 2], [1, 5]]) # точный тест фишера
    return anova, oddsratio, pvalue, res_tt, res_mw, res_kw, res_chi, res_pearson, res_spear, res1_ks, res2_ks, shapiro_test

In [5]:
df_no_ind = df[df['Ab_ind'] ==0]
df_ind = df[df['Ab_ind'] ==1]

In [14]:
for i in columns:
  lst1 = df_no_ind[i]
  lst2 = df_ind[i]
  res_tt = ttest_ind(lst1, lst2)
  if res_tt[1] < .05:
    print(i,  res_tt[1])
  else:
    pass

Min_Platlets 2.221286963984418e-07
Max_ESR 0.0001178458266501955
Max_Fibrinogen 8.00987160171709e-08
Max_Leucocytes 0.018171946741101418
Max_CRP 6.478105938120063e-05
Severity 2.6819834628561046e-05
IHD 2.695662569637109e-05
COPD 1.4047360314090377e-14
ALT 0.016309709524600507
duration 2.6351622957897995e-10
Age 0.04133697351902121
RF_0.0 0.0015310975677618602
RF_1.0 2.402749193545252e-06
RF_2.0 0.0001053376798051839
Ab_ind 0.0
L/Pl_Ratio 0.005649827499620999


In [15]:
for i in columns:
  lst1 = df_no_ind[i]
  lst2 = df_ind[i]
  shapiro_test = stats.shapiro(lst1)
  shapiro_test2 = stats.shapiro(lst2)
  if shapiro_test[1] < .05:
    print(i,  shapiro_test[1])
  if shapiro_test2[1] < .05:
    print(i,  shapiro_test2[1])
  else:
    pass

Unnamed: 0 6.80880455098884e-14
Unnamed: 0 1.0859444046218414e-05
Min_Platlets 2.2496317308995373e-11
Min_Platlets 0.0046564731746912
Max_ESR 2.552094656493864e-06
Max_ESR 2.1818234017700888e-05
Max_Fibrinogen 8.701200213717984e-17
Max_Fibrinogen 4.6909113734727725e-05
Max_LDH 1.7956045894607087e-07
Max_LDH 0.0016341493465006351
Max_Leucocytes 1.087012632683404e-10
Max_Leucocytes 1.7819214159775498e-10
Gender 2.4788868492001074e-36
Gender 9.546117563837618e-18
Max_CRP 5.4264262484617843e-17
Max_CRP 3.7895981491731234e-11
Severity 2.690117458673659e-36
Severity 8.67677927777552e-19
IHD 2.61933435317887e-36
IHD 5.795048493739047e-19
COPD 0.0
COPD 4.362893458973837e-21
Bronchial_asthma 0.0
Bronchial_asthma 7.012660368243035e-26
Obesity 0.0
Obesity 3.116403017944176e-26
Diabetes 0.0
Diabetes 1.20135796737943e-26
ALT 9.310547806128627e-29
ALT 1.2634226974214896e-16
AST 8.480640728205856e-33
AST 2.0524005358665723e-18
duration 4.1189124333484983e-10
duration 1.825566141633317e-05
Age 5.65343

/usr/local/lib/python3.6/dist-packages/scipy/stats/morestats.py:1673: UserWarning: Input data for shapiro has range zero. The results may not be accurate.
  warnings.warn("Input data for shapiro has range zero. The results "


In [21]:
for i in columns:
  lst1 = df_no_ind[i]
  lst2 = df_ind[i]
  res_kw = kruskal(lst1, lst2)## краскела уолиса
  
  if res_kw[1] < .05:
    print(i,  res_kw[1])
  else:
    pass

Min_Platlets 6.565008618092226e-07
Max_ESR 0.00013135466386457627
Max_Fibrinogen 1.9387907471511455e-05
Max_Leucocytes 0.004054041791523019
Max_CRP 2.8435491579278232e-06
Severity 2.900583157734838e-05
IHD 2.9151073942774236e-05
COPD 3.676413140899496e-14
ALT 1.054902874670756e-09
AST 0.017697926723581083
duration 5.266070909595151e-10
Age 0.007968861256725126
RF_0.0 0.001565528638204953
RF_1.0 2.7323340211092217e-06
RF_2.0 0.00011134605344588802
Ab_ind 1.7896263240240073e-189
L/Pl_Ratio 7.433117108337258e-05


In [20]:
for i in columns:
  lst1 = df_no_ind[i]
  lst2 = df_ind[i]
  anova = stats.f_oneway(lst1, lst2)## краскела уолиса
  
  if anova[1] < .05:
    print(i,  anova[1])
  else:
    pass


Min_Platlets 2.2212869639839813e-07
Max_ESR 0.00011784582665018135
Max_Fibrinogen 8.009871601715198e-08
Max_Leucocytes 0.018171946741098028
Max_CRP 6.478105938119224e-05
Severity 2.6819834628556194e-05
IHD 2.695662569636545e-05
COPD 1.4047360314088254e-14
ALT 0.016309709524597274
duration 2.6351622957894273e-10
Age 0.041336973519015996
RF_0.0 0.0015310975677616065
RF_1.0 2.4027491935449034e-06
RF_2.0 0.00010533767980516619
Ab_ind 0.0
L/Pl_Ratio 0.005649827499620078


In [ ]:
res_pearson = pearsonr(lst_1, lst_2)# корреляция пирсона
    res_spear = spearmanr(lst_1, lst_2)

In [25]:
def pipeline_gbm_grid(data, target_column, cols_to_remove, split_ratio):
    !pip install h2o
    import h2o
    h2o.init()
    from h2o.estimators.gbm import H2OGradientBoostingEstimator
    from h2o.grid.grid_search import H2OGridSearch
    data = h2o.H2OFrame(data)
    predictors = data.col_names
    predictors.remove(target_column)
    data[target_column] = data[target_column].asfactor()
    #data = data.drop(cols_to_remove)
    train, test = data.split_frame(seed = 12345, ratios = [split_ratio])
    target = target_column
    #predictors = data.col_names.remove(target_column)
    hyperparams_tune = {
    'max_depth' : list(range(2, 6, 1)),
    'sample_rate' : [x/100. for x in range(20,101)],
    'col_sample_rate' : [x/100. for x in range(20,101)],
    'min_split_improvement': [0, 1e-8, 1e-6, 1e-4]}

    search_criteria_tune = {
    'strategy' : "RandomDiscrete",
    'max_runtime_secs' : 300,  ## limit the runtime to 5 minutes
    'max_models' : 15,  ## build no more than 15 models
    'stopping_rounds' : 5,
    'stopping_metric' : "AUC",
    'stopping_tolerance': 1e-3}
    gbm_final_grid = H2OGradientBoostingEstimator(
    ## More trees is better if the learning rate is small enough
    ## we use "more than enough" trees with early stopping
    ntrees = 10000,
    ## we use learn rate with annealing for a fast scan
    learn_rate = 0.05,
    learn_rate_annealing = 0.99,
    ## Score every 10 trees to make early stopping reproducible
    score_tree_interval = 10,
    ## Set seed for reproducibility
    seed = 12345)
    final_grid = H2OGridSearch(
    gbm_final_grid,
    hyper_params = hyperparams_tune,
    grid_id = 'gbm_final_grid',
    search_criteria = search_criteria_tune)
    final_grid.train(
    x = predictors,
    y = target,
    max_runtime_secs = 180, # can override setting in search_criteria_tune
    training_frame = train)
    #validation_frame = test)
    
    
    sorted_gbm_grid = final_grid.get_grid(sort_by = "auc", decreasing = True)
    best_gbm = sorted_gbm_grid.models[0]
    #best_gbm_perf_valid = best_gbm.model_performance(valid)
    best_gbm_perf_test = best_gbm.model_performance(test)
    return best_gbm_perf_test
    #final_grid.get_grid(sort_by = "auc", decreasing = True), 

In [27]:
pipeline_gbm_grid(data = df, target_column = 'Ab_ind', cols_to_remove = 'Ab_ind', split_ratio = .8)

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,3 mins 47 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.0.4
H2O_cluster_version_age:,19 days
H2O_cluster_name:,H2O_from_python_unknownUser_zxvatj
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.180 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


Parse progress: |█████████████████████████████████████████████████████████| 100%
gbm Grid Build progress: |████████████████████████████████████████████████| 100%

ModelMetricsBinomial: gbm
** Reported on test data. **

MSE: 0.10925503006900682
RMSE: 0.33053748663201094
LogLoss: 0.3636593540919993
Mean Per-Class Error: 0.2690981432360743
AUC: 0.7692307692307692
AUCPR: 0.44469544749897755
Gini: 0.5384615384615383

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.15665475327590841: 


,,0,1,Error,Rate
0,0,115.0,30.0,0.2069,(30.0/145.0)
1,1,9.0,17.0,0.3462,(9.0/26.0)
2,Total,124.0,47.0,0.2281,(39.0/171.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.156655,0.465753,46.0
1,max f2,0.136284,0.586420,57.0
2,max f0point5,0.463992,0.512821,12.0
3,max accuracy,0.684701,0.871345,5.0
4,max precision,0.872231,1.000000,0.0
5,max recall,0.021689,1.000000,155.0
6,max specificity,0.872231,1.000000,0.0
7,max absolute_mcc,0.463992,0.370140,12.0
8,max min_per_class_accuracy,0.136284,0.730769,57.0
9,max mean_per_class_accuracy,0.136284,0.730902,57.0



Gains/Lift Table: Avg response rate: 15.20 %, avg score: 14.41 %


,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain,kolmogorov_smirnov
0,1,0.011696,0.784974,3.288462,3.288462,0.500000,0.861295,0.500000,0.861295,0.038462,0.038462,228.846154,228.846154,0.031565
1,2,0.023392,0.716908,6.576923,4.932692,1.000000,0.746170,0.750000,0.803732,0.076923,0.115385,557.692308,393.269231,0.108488
2,3,0.035088,0.681994,6.576923,5.480769,1.000000,0.686945,0.833333,0.764803,0.076923,0.192308,557.692308,448.076923,0.185411
3,4,0.040936,0.649972,0.000000,4.697802,0.000000,0.657635,0.714286,0.749494,0.000000,0.192308,-100.000000,369.780220,0.178515
4,5,0.052632,0.509034,3.288462,4.384615,0.500000,0.582819,0.666667,0.712455,0.038462,0.230769,228.846154,338.461538,0.210080
5,6,0.105263,0.387925,2.192308,3.288462,0.333333,0.446686,0.500000,0.579570,0.115385,0.346154,119.230769,228.846154,0.284085
6,7,0.152047,0.252080,0.822115,2.529586,0.125000,0.306707,0.384615,0.495612,0.038462,0.384615,-17.788462,152.958580,0.274271
7,8,0.204678,0.202360,2.923077,2.630769,0.444444,0.223902,0.400000,0.425744,0.153846,0.538462,192.307692,163.076923,0.393634
8,9,0.304094,0.144384,1.160633,2.150148,0.176471,0.166243,0.326923,0.340907,0.115385,0.653846,16.063348,115.014793,0.412467
9,10,0.403509,0.102322,0.773756,1.811037,0.117647,0.122147,0.275362,0.287010,0.076923,0.730769,-22.624434,81.103679,0.385942


In [39]:
def all_estimators_pipline(data, predictors, target, test_size, accuracy_list = []):
    from sklearn.metrics import roc_auc_score
    x = data[predictors]
    y = data[target]
    x_train,x_test,y_train,y_test = train_test_split(x,y, test_size=test_size, random_state=2)
    #Logistic Regression
    log_reg = LogisticRegression()
    log_reg.fit(x_train, y_train)
    log_reg_pred = log_reg.predict(x_test)
    log_reg_acc = accuracy_score(y_test, log_reg_pred)
    accuracy_list.append(100*log_reg_acc)
    roc_auc_score(y_test,  log_reg.predict_proba(x_test)[:, 1])
    print(Fore.GREEN + "AUC of Logistic Regression is : ", "{:.2f}%".format(roc_auc_score(y_test,  log_reg.predict_proba(x_test)[:, 1])))
    
    #svm
    sv_clf = SVC()
    sv_clf.fit(x_train, y_train)
    sv_clf_pred = sv_clf.predict(x_test)
    sv_clf_acc = accuracy_score(y_test, sv_clf_pred)
    accuracy_list.append(100* sv_clf_acc)
    #roc_auc_score(y_test,  sv_clf.predict_proba(x_test)[:, 1],probability=True)
    print(Fore.GREEN + "Accuracy of SVC is : ", "{:.2f}%".format(100* sv_clf_acc))
    
    # K Neighbors Classifier

    kn_clf = KNeighborsClassifier(n_neighbors=6)
    kn_clf.fit(x_train, y_train)
    kn_pred = kn_clf.predict(x_test)
    kn_acc = accuracy_score(y_test, kn_pred)
    accuracy_list.append(100*kn_acc)
    roc3 =  roc_auc_score(y_test,  kn_clf.predict_proba(x_test)[:, 1])
    print(Fore.GREEN + "AUC of K Neighbors Classifier is : ", "{:.2f}%".format(roc3))
    # Decision Tree Classifier

    dt_clf = DecisionTreeClassifier(max_leaf_nodes=3, random_state=0, criterion='entropy')
    dt_clf.fit(x_train, y_train)
    dt_pred = dt_clf.predict(x_test)
    dt_acc = accuracy_score(y_test, dt_pred)
    accuracy_list.append(100*dt_acc)
    print(Fore.GREEN + "Accuracy of Decision Tree Classifier is : ", "{:.2f}%".format(100* dt_acc))
    
    # RandomForestClassifier

    r_clf = RandomForestClassifier(max_features=0.5, max_depth=15, random_state=1)
    r_clf.fit(x_train, y_train)
    r_pred = r_clf.predict(x_test)
    r_acc = accuracy_score(y_test, r_pred)
    roc_ = roc_auc_score(y_test,  r_clf.predict_proba(x_test)[:, 1])
    accuracy_list.append(100*r_acc)
    print(Fore.GREEN + "AUC of Random Forest Classifier is : ", "{:.2f}%".format(roc_))
    
    
    #xgb
    gradientboost_clf = GradientBoostingClassifier(max_depth=2, random_state=1)
    gradientboost_clf.fit(x_train,y_train)
    gradientboost_pred = gradientboost_clf.predict(x_test)
    gradientboost_acc = accuracy_score(y_test, gradientboost_pred)
    #accuracy_list.append(100*gradientboost_acc)
    roc = roc_auc_score(y_test,  gradientboost_clf.predict_proba(x_test)[:, 1])
    print(Fore.GREEN + "AUC of Gradient Boosting is : ", "{:.2f}%".format(roc))
    
    #  lightgbm 

    lgb_clf = lightgbm.LGBMClassifier(max_depth=2, random_state=4)
    lgb_clf.fit(x_train,y_train)
    lgb_pred = lgb_clf.predict(x_test)
    lgb_acc = accuracy_score(y_test, lgb_pred)
    accuracy_list.append(100*lgb_acc)
    roc2 = roc_auc_score(y_test,  lgb_clf.predict_proba(x_test)[:, 1])
    print(Fore.GREEN + "AUC of LGBMClassifier is : ","{:.2f}%".format(roc2))
    
    # CatBoostClassifier

    # cat_clf = CatBoostClassifier()
    # cat_clf.fit(x_train,y_train)
    # cat_pred = cat_clf.predict(x_test)
    # cat_acc = accuracy_score(y_test, cat_pred)
    # accuracy_list.append(100*cat_acc)
    # roc4 = roc_auc_score(y_test,  cat_clf.predict_proba(x_test)[:, 1])
    # print(Fore.GREEN + "AUC of CatBoostClassifier is : ","{:.2f}%".format(roc4))

In [30]:
 columns

Index(['Unnamed: 0', 'Min_Platlets', 'Max_ESR', 'Max_Fibrinogen', 'Max_LDH',
       'Max_Leucocytes', 'Gender', 'Max_CRP', 'Severity', 'IHD', 'COPD',
       'Bronchial_asthma', 'Obesity', 'Diabetes', 'ALT', 'AST', 'duration',
       'Age', 'Max_Creatinine', 'RF_0.0', 'RF_1.0', 'RF_2.0', 'RF_3.0', 'GFR',
       'Ab_ind', 'L/Pl_Ratio'],
      dtype='object')

In [31]:
predictors = ['Min_Platlets', 'Max_ESR', 'Max_Fibrinogen', 'Max_LDH',
       'Max_Leucocytes', 'Gender', 'Max_CRP', 'Severity', 'IHD', 'COPD',
       'Bronchial_asthma', 'Obesity', 'Diabetes', 'ALT', 'AST', 'duration',
       'Age', 'Max_Creatinine', 'RF_0.0', 'RF_1.0', 'RF_2.0', 'RF_3.0', 'GFR',
        'L/Pl_Ratio']

In [34]:
import matplotlib.pyplot as plt
import warnings
import seaborn as sns
from colorama import Fore, Back, Style 
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from mlxtend.plotting import plot_confusion_matrix
from plotly.offline import plot, iplot, init_notebook_mode
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.express as px
from statsmodels.formula.api import ols
import plotly.graph_objs as gobj

init_notebook_mode(connected=True)
warnings.filterwarnings("ignore")
import plotly.figure_factory as ff

%matplotlib inline

import xgboost
import lightgbm
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
!pip install catboost
from catboost import CatBoostClassifier

     |████████████████████████████████| 65.8MB 73kB/s 


In [40]:
all_estimators_pipline(df, predictors = predictors , target = 'Ab_ind', test_size = 0.2, accuracy_list = [])

AUC of Logistic Regression is :  0.75%
Accuracy of SVC is :  82.08%
AUC of K Neighbors Classifier is :  0.62%
Accuracy of Decision Tree Classifier is :  82.08%
AUC of Random Forest Classifier is :  0.81%
AUC of Gradient Boosting is :  0.78%
AUC of LGBMClassifier is :  0.79%


In [42]:
def pipeline_auto_ml(data, target_column, cols_to_remove, split_ratio):
    """Returns default gbm perfomance on H2O"""
#     !pip install h2o
    import h2o
    h2o.init()
    from h2o.automl import H2OAutoML
    data = h2o.H2OFrame(data)
    predictors = data.col_names
    predictors.remove(target_column)
    data[target_column] = data[target_column].asfactor()
    train, test = data.split_frame(seed = 12345, ratios 
                                   = [split_ratio])
    target = target_column
    #predictors = data.col_names.remove(target_column)
    aml = H2OAutoML(max_models = 6, max_runtime_secs = 
                    120, seed = 12345)
    aml.train(x = predictors, y = target, 
              training_frame = train, leaderboard_frame = test)
    lb = aml.leaderboard
    return lb.head(rows=5)

In [43]:
pipeline_auto_ml(data = df, target_column = 'Ab_ind', cols_to_remove = 'Ab_ind', split_ratio = .8)

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,13 mins 00 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.0.4
H2O_cluster_version_age:,19 days
H2O_cluster_name:,H2O_from_python_unknownUser_zxvatj
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.151 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


Parse progress: |█████████████████████████████████████████████████████████| 100%
AutoML progress: |████████████████████████████████████████████████████████| 100%


model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
GLM_1_AutoML_20210220_190523,0.760477,0.369492,0.451736,0.314058,0.333191,0.111016
XGBoost_3_AutoML_20210220_190523,0.755836,0.391504,0.366168,0.308621,0.345077,0.119078
XGBoost_2_AutoML_20210220_190523,0.740053,0.382041,0.407312,0.307162,0.33763,0.113994
XGBoost_1_AutoML_20210220_190523,0.735279,0.416167,0.331696,0.26565,0.350204,0.122643
StackedEnsemble_AllModels_AutoML_20210220_190523,0.693634,0.384082,0.356584,0.342175,0.338576,0.114633
